In [1]:
import cProfile
import earthaccess
import numpy as np
import s3fs
import xarray as xr
import h5py

try:
    import h5coro
except:
    !mamba install -c conda-forge h5coro --yes
    import h5coro

from h5coro import h5coro, s3driver, filedriver
h5coro.config(errorChecking=True, verbose=False, enableAttributes=False)

# Hi!

This is a template for benchmarking 2 workflows.

# Benchmarking XX Format

We will benchmark the following workflows using the XX format.

[ADD A BIT OF RELEVANT INFORMATION ABOUT THAT FORMAT]

1. Reading 1 full group (`h_ph`)
2. Spatially subsetting that group (tbd)

via the following libraries: h5py, h5coro and xarray

### Setup Steps

NOTE, below we use the `original` directory, this should be replaced if you are working with a different dataset.

In [ ]:
# Checkout the files
# !mamba install -c conda-forge awscli -y
!aws s3 ls s3://nasa-cryo-scratch/h5cloud/original/

In [ ]:
# Set the variables
bucket = 'nasa-cryo-scratch'
directory = 'h5cloud/original/'
group = '/gt2l/heights'
variable = 'h_ph'

In [ ]:
# Create a list of s3 objects
s3 = s3fs.S3FileSystem(anon=False)

# This generates a list of strings with filenames
s3path = f's3://{bucket}/{directory}*'
remote_files = [f's3://{path}' for path in s3.glob(s3path)]
remote_files

# Workflow 1 - Read full group

## Option 1: Read the group with `h5py`

In [ ]:
%%time
arrays = []
for file in remote_files:
    with h5py.File(s3.open(file, 'rb')) as f:
        data = f[f'{group}/{variable}'][:]
        arrays.append(data)
final_array = np.concatenate(arrays)

In [ ]:
%%time
assert final_array.shape == (298271231,)
final_array.mean()

## Option 2: Read the group with `xarray`

In [ ]:
%%time
s3_fileset = [s3.open(file) for file in remote_files]
ds = xr.open_mfdataset(s3_fileset, group=group, combine='by_coords', engine='h5netcdf')

In [ ]:
%%time
assert ds[variable].shape == (298271231,)
ds[variable].values.mean()

## Option 3: Read the group with `h5coro`

# Workflow 2 - Spatially Subset

## Option 1: Spatially subset with `h5py`

## Option 2: Spatially subset with `xarray`

## Option 3: Spatially subset with `h5coro`